In [1]:
import pandas as pd

/var/folders/gm/pxzh4m3d3xlbcffhxnmwc3jm0000gn/T/ipykernel_79754/2035458946.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv("data/generated/34284.csv")
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34284 entries, 0 to 34283
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           34284 non-null  int64  
 1   mmsi                 34284 non-null  float64
 2   timestamp            34284 non-null  float64
 3   distance_from_shore  34284 non-null  float64
 4   distance_from_port   34284 non-null  float64
 5   speed                34284 non-null  float64
 6   course               34284 non-null  float64
 7   lat                  34284 non-null  float64
 8   lon                  34284 non-null  float64
 9   is_fishing           34284 non-null  float64
dtypes: float64(9), int64(1)
memory usage: 2.6 MB
None


In [5]:
groups = data.groupby("mmsi")
for group in groups:
    print(group)

(1252339803566.0,       Unnamed: 0          mmsi     timestamp  distance_from_shore  \
0              0  1.252340e+12  1.325376e+09                  0.0   
1              1  1.252340e+12  1.325378e+09                  0.0   
2              2  1.252340e+12  1.325379e+09                  0.0   
3              3  1.252340e+12  1.325380e+09                  0.0   
4              4  1.252340e+12  1.325381e+09                  0.0   
...          ...           ...           ...                  ...   
5709        5709  1.252340e+12  1.344132e+09                  0.0   
5710        5710  1.252340e+12  1.344133e+09                  0.0   
5711        5711  1.252340e+12  1.344134e+09                  0.0   
5712        5712  1.252340e+12  1.344135e+09                  0.0   
5713        5713  1.252340e+12  1.344136e+09                  0.0   

      distance_from_port  speed  course        lat       lon  is_fishing  
0                    0.0    0.0   153.0  52.458649  4.581200        -1.0  
1  